In [13]:
# Modules to be imported in order
import warnings
warnings.filterwarnings("ignore")
import gmsh
gmsh.initialize()

# Additional module
import numpy as np
from dolfinx import io
from dolfinx.io import gmshio
from mpi4py import MPI

gmsh.model.add("CY3D")
ro, ri, H = 0.3, 0.2, 10

cylinder_o = gmsh.model.occ.addCylinder(0, 0, 0, 0, 0, H, ro)
cylinder_i = gmsh.model.occ.addCylinder(0, 0, 0, 0, 0, H, ri)

solid = gmsh.model.occ.cut([(3, cylinder_o)], [(3, cylinder_i)])
gmsh.model.occ.synchronize()

# Physical entity: volume
volumes = gmsh.model.getEntities(dim=3)
assert(volumes == solid[0])
solid_marker = 11
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], solid_marker)
gmsh.model.setPhysicalName(volumes[0][0], solid_marker, "Solid volume")

# Physical entity: surface
# For the surfaces, we start by finding all surfaces, and then compute the geometrical center such that we can indentify which are inlets, outlets, walls and the obstacle.
# As the walls will consist of multiple surfaces, and the obstacle is circular, we need to find all entites before addin the physical group.
surfaces = gmsh.model.occ.getEntities(dim=2)
bottom_marker, top_marker, outer_wall_marker, inner_wall_marker = 1, 3, 5, 7
outer_wall = []
inner_wall = []
for surface in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
    if np.allclose(com, [0, 0, 0]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], bottom_marker)
        bottom = surface[1]
        gmsh.model.setPhysicalName(surface[0], bottom_marker, "Bottom")
    elif np.allclose(com, [0, 0, H]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], top_marker)
        gmsh.model.setPhysicalName(surface[0], top_marker, "Top")
    elif np.isclose(com[2], 0) or np.isclose(com[1], ro) or np.isclose(com[2], H) or np.isclose(com[1],0):
        outer_wall.append(surface[1])
    else:
        inner_wall.append(surface[1])
gmsh.model.addPhysicalGroup(2, outer_wall, outer_wall_marker)
gmsh.model.setPhysicalName(2, outer_wall_marker, "Walls")
gmsh.model.addPhysicalGroup(2, inner_wall, inner_wall_marker)
gmsh.model.setPhysicalName(2, inner_wall_marker, "Obstacle")

distance = gmsh.model.mesh.field.add("Distance")
gmsh.model.mesh.field.setNumbers(distance, "FacesList", inner_wall)

r = 0.1
resolution = 0.01
threshold = gmsh.model.mesh.field.add("Threshold")
gmsh.model.mesh.field.setNumber(threshold, "IField", distance)
gmsh.model.mesh.field.setNumber(threshold, "LcMin", resolution)
gmsh.model.mesh.field.setNumber(threshold, "LcMax", 20*resolution)
gmsh.model.mesh.field.setNumber(threshold, "DistMin", 0.5*r)
gmsh.model.mesh.field.setNumber(threshold, "DistMax", r)

inlet_dist = gmsh.model.mesh.field.add("Distance")
gmsh.model.mesh.field.setNumbers(inlet_dist, "FacesList", [bottom])
inlet_thre = gmsh.model.mesh.field.add("Threshold")
gmsh.model.mesh.field.setNumber(inlet_thre, "IField", inlet_dist)
gmsh.model.mesh.field.setNumber(inlet_thre, "LcMin", 5*resolution)
gmsh.model.mesh.field.setNumber(inlet_thre, "LcMax", 10*resolution)
gmsh.model.mesh.field.setNumber(inlet_thre, "DistMin", 0.1)
gmsh.model.mesh.field.setNumber(inlet_thre, "DistMax", 0.5)

minimum = gmsh.model.mesh.field.add("Min")
gmsh.model.mesh.field.setNumbers(minimum, "FieldsList", [threshold, inlet_thre])
gmsh.model.mesh.field.setAsBackgroundMesh(minimum)

gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

gmsh.write("pipe3D.msh")

domain, __, facet_tag = gmshio.read_from_msh("pipe3D.msh", MPI.COMM_WORLD, gdim = 3)
# The solution can be written to a file, and visualize it with ParaView or VisIt:
with io.XDMFFile(domain.comm, "pipe3D.xdmf", "w") as file:
    file.write_mesh(domain)
    file.write_meshtags(facet_tag)


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Circle)
Info    : [ 50%] Meshing curve 4 (Circle)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : [ 90%] Meshing curve 6 (Line)
Info    : Done meshing 1D (Wall 0.00228746s, CPU 0.002588s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 4 (Cylinder, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.243133s, CPU 0.234591s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 4256 nodes...
Info    : Done tetrahedrizing 4264 nodes (Wall 0.0655056s, CPU 0.065561s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
I